# カスタムコンテナハンズオン

インターネットにつながる環境でコンテナを作り、インターネットにつながらないSandbox環境でモデル開発をする方法のハンズオンをします。

本ハンズオンでは scikit-learnを使い 決定木(decision tree)のアルゴリズムを例にします。

- [scikit-learn](http://scikit-learn.org/stable/)

- [decision tree](http://scikit-learn.org/stable/modules/tree.html)


# Part 1: SageMakerのNotebookを使って学習コンテナの作成と登録

LandingVPCインターネットGatewayを有し、NATを使ってインターネット上のライブラリーをダウンロードすることができます。
Sandbox環境ではインターネットにアクセスできないため、このLandingVPC環境で学習に必要なライブラリーを備えたdockerコンテナを作り、ECRに登録します。

### SageMakerによるDocker container の実行

学習と推論は同じコンテナイメージを使います。、Amazon SageMaker はコンテナに `train` と `serve`のコマンドを渡して、学習と推論を使い分けています。具的には次の 2 つのコマンドを使用して、イメージを実行します。

`docker run <イメージ> train`

`docker run <イメージ> serve`



#### 学習コンテナについて

学習コンテナは　`/opt/ml` 以下のつぎのディレクトリ構成にあるファイルを利用します。

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

##### input

* `/opt/ml/input/config`には、設定ファイルが配置されます。 `hyperparameters.json` は、ハイパーパラメーターをJSON形式で記述します。`resourceConfig.json` は、分散学習に使います。scikit-learnでは分散学習をサポートしていないので、ここでは無視されます。
* `/opt/ml/input/data/<channel_name>/` (for File mode) 入力データです。  channels は `CreateTrainingJob`を呼ぶと作られます。S3から channelのディレクトリにファイルがコピーされます。

#####  output

* `/opt/ml/model/` 学習されたモデルファイルの出力先です。tar と gzip 形式で圧縮されてアーカイブされます。
* `/opt/ml/output` デバッグとトラブルシューティング用の詳細がログとして記録されます。成功しているときは、無視してください。

#### 推論について

推論ホスト用コンテナが、推論のHTTPリクエストに返答します。推論は RESTful APIが呼び出されます。　ハンズオンの例では Python のマイクロフレームワークである Flask を使用します。

![Request serving stack](stack.png)


Amazon SageMaker 推論エンドポイントは 2つリクエストの応答を提供します。

* `/ping` の `GET` を受信すると、推論エンドポイントがアクティブな時は 200を返します。 
* `/invocations` の `POST` リクエストを受信するとアルゴリズムの応答を返します。


推論コンテナのディレクトリ配置は学習コンテナと同じ以下となります。

    /opt/ml
    └── model
        └── <model files>



### コンテナを作成するファイル

`container` ディレクトリにコンテナイメージ作成に必要な全てのファイルがあります。

    .
    ├── Dockerfile
    ├── build_and_push.sh
    └── decision_trees
        ├── nginx.conf
        ├── predictor.py
        ├── serve
        ├── train
        └── wsgi.py

各ファイルの説明:

* __`Dockerfile`__ コンテナイメージをビルドする際の設定ファイル 
* __`build_and_push.sh`__ コンテナイメージのビルドとECRのリポジトリへのプッシュを行うスクリプト 
* __`decision_trees`__ 学習時に実行されるスクリプト

* __`nginx.conf`__ Nginx用の設定ファイル
* __`predictor.py`__ gunicornから呼び出されるアプリケーションが記述されたスクリプトです。 
* __`serve`__ 推論エンドポイント作成時に実行されるスクリプト 
* __`train`__  学習時に実行されるスクリプト 
* __`wsgi.py`__  gunicornがリクエストを受けた際にFlaskアプリを起動するファイル 

`train` and `predictor.py`をアプリケーションに合わせて変更することで動作します。


### Dockerfileについて

Dockerfileに、必要なパッケージやアプリ、各種設定をすることで必要なDockerイメージを作成することができます。

Pythonの学習スタックを考慮して、標準的な Ubuntu環境に scikit-learnに必要なツールをインストールするように Dockerファイルに記載しています。

まずは、Dockerfileの例を見てましょう。


In [4]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn && \
        (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../..

### コンテナイメージの作成


コンテナイメージを`docker build`で作成します。そのイメージを ECR に`docker push`で登録します。



In [5]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/9 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
f7277927d38a: Pulling fs layer
8d3eac894db4: Pulling fs layer
edf72af6d627: Pulling fs layer
3e4f86211d23: Pulling fs layer
3e4f86211d23: Waiting
edf72af6d627: Verifying Checksum
edf72af6d627: Download complete
8d3eac894db4: Verifying Checksum
8d3eac894db4: Download complete
f7277927d38a: Verifying Checksum
f7277927d38a: Download complete
3e4f86211d23: Verifying Checksum
3e4f86211d23: Download complete
f7277927d38a: Pull complete
8d3eac894db4: Pull complete
edf72af6d627: Pull complete
3e4f86211d23: Pull complete
Digest: sha256:97b54e5692c27072234ff958a7442dde4266af21e7b688e7fca5dc5acc8ed7d9
Status: Downloaded newer image for ubuntu:16.04
 ---> 5e13f8dd4c1a
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Running in 9a05c8071a33
Removing intermediate container 9a05c8071a33
 ---> d53ceb255919
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### 関連ファイルをS3にアップロード
Sandbox環境にデータのnotebookを送るため、S3にアップロードします。



In [6]:
import sagemaker as sage

prefix = 'LAB-handson'
SRC_DIRECTORY = '../scikit_custom'

sess = sage.Session()

src_location = sess.upload_data(SRC_DIRECTORY, key_prefix=prefix)

In [7]:
print(src_location)

s3://sagemaker-ap-northeast-1-925889618331/LAB-handson
